In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(13473, 128, 128, 3)

In [8]:
y.shape

(13473, 7)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(10778, 128, 128, 3)

In [11]:
X_test.shape

(2695, 128, 128, 3)

In [12]:
y_train.shape

(10778, 7)

In [13]:
y_test.shape

(2695, 7)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

7

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 12:13:13.141355: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


337/337 [==============================] - 139s 409ms/step - loss: 0.7053 - accuracy: 0.7493 - val_loss: 0.4559 - val_accuracy: 0.8315
Epoch 2/10
337/337 [==============================] - 153s 453ms/step - loss: 0.3053 - accuracy: 0.8950 - val_loss: 0.7058 - val_accuracy: 0.7607
Epoch 3/10
337/337 [==============================] - 126s 373ms/step - loss: 0.1969 - accuracy: 0.9344 - val_loss: 0.3465 - val_accuracy: 0.8865
Epoch 4/10
337/337 [==============================] - 82s 244ms/step - loss: 0.1595 - accuracy: 0.9435 - val_loss: 0.3781 - val_accuracy: 0.8716
Epoch 5/10
337/337 [==============================] - 84s 249ms/step - loss: 0.1192 - accuracy: 0.9582 - val_loss: 1.0494 - val_accuracy: 0.7373
Epoch 6/10
337/337 [==============================] - 88s 261ms/step - loss: 0.1143 - accuracy: 0.9613 - val_loss: 0.5240 - val_accuracy: 0.8286
Epoch 7/10
337/337 [==============================] - 104s 308ms/step - loss: 0.0822 - accuracy: 0.9713 - val_loss: 0.1205 - val_accuracy:

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 79.55%


In [19]:
cnn_scores

[0.7307010889053345, 0.7955473065376282]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

85/85 [==============================] - 10s 108ms/step
Accuracy: 79.55%
Precision: 88.22%
Recall: 79.55%
F1 score: 80.37%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-apel+kentang.csv', index=False)

+------+----------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                   | True Labels              | Predicted Labels         |
|------+----------------------------------------------------------------------------+--------------------------+--------------------------|
|    0 | 3fb3806c-db88-491d-b9b0-191b3d5964cd___FREC_C.Rust 4130_90deg.JPG          | Apple___Cedar_apple_rust | Apple___Cedar_apple_rust |
|    1 | b536ea91-c5f8-413b-87f3-a22ac3d959d1___RS_HL 1825_180deg.JPG               | Potato___healthy         | Potato___healthy         |
|    2 | 9a82c830-7854-4f7f-84c5-f7414cd238b0___FREC_Scab 3069_new30degFlipLR.JPG   | Apple___Apple_scab       | Apple___healthy          |
|    3 | 2551a3b8-b5dd-4716-88e6-517a012a44f1___RS_HL 7551.JPG                      | Apple___healthy          | Apple___healthy          |
|    4 | 68fdb88e-75

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-apel+kentang.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                 | True Labels              | Predicted Labels         |
|------+--------------------------------------------------------------------------+--------------------------+--------------------------|
|    2 | 9a82c830-7854-4f7f-84c5-f7414cd238b0___FREC_Scab 3069_new30degFlipLR.JPG | Apple___Apple_scab       | Apple___healthy          |
|    5 | 0395b847-2c73-4674-826f-33a6afb5b4fe___FREC_Scab 3287.JPG                | Apple___Apple_scab       | Apple___healthy          |
|    8 | 4a50158d-dcd0-4a1f-885e-2d9b16a4e344___FREC_Scab 3205_new30degFlipLR.JPG | Apple___Apple_scab       | Apple___healthy          |
|    9 | 7050bb1d-af0d-47c4-8f35-23b9561b8024___RS_LB 3313_180deg.JPG             | Potato___Late_blight     | Apple___healthy          |
|   12 | a31b79

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

337/337 [==============================] - 45s 134ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 957.2753190994263 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

85/85 [==============================] - 10s 121ms/step
Random Forest Classifier Accuracy: 96.55%
Random Forest Classifier Precision: 0.97
Random Forest Classifier Recall: 0.97
Random Forest Classifier F1 Score: 0.97


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-apel+kentang.csv', index=False)

+------+----------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                   | True Labels              | Predicted Labels         |
|------+----------------------------------------------------------------------------+--------------------------+--------------------------|
|    0 | 3fb3806c-db88-491d-b9b0-191b3d5964cd___FREC_C.Rust 4130_90deg.JPG          | Apple___Cedar_apple_rust | Apple___Cedar_apple_rust |
|    1 | b536ea91-c5f8-413b-87f3-a22ac3d959d1___RS_HL 1825_180deg.JPG               | Potato___healthy         | Potato___healthy         |
|    2 | 9a82c830-7854-4f7f-84c5-f7414cd238b0___FREC_Scab 3069_new30degFlipLR.JPG   | Apple___Apple_scab       | Apple___Apple_scab       |
|    3 | 2551a3b8-b5dd-4716-88e6-517a012a44f1___RS_HL 7551.JPG                      | Apple___healthy          | Apple___healthy          |
|    4 | 68fdb88e-75

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-apel+kentang.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                   | True Labels              | Predicted Labels         |
|------+----------------------------------------------------------------------------+--------------------------+--------------------------|
|   31 | b5a75c29-c79a-4db0-8976-e33e48628ccb___FREC_C.Rust 9966.JPG                | Apple___Cedar_apple_rust | Apple___Apple_scab       |
|   45 | 8df36a92-b562-4bff-975b-ff08d8da40f6___FREC_Scab 3119.JPG                  | Apple___Apple_scab       | Apple___Cedar_apple_rust |
|   93 | 7eee2c9b-285a-411a-950b-7cd5547a9440___JR_FrgE.S 3028.JPG                  | Apple___Black_rot        | Apple___Cedar_apple_rust |
|  122 | 201186ce-cec5-47ef-9f88-bd8c4abab2c8___RS_LB 2813_180deg.JPG               | Potato___Late_blight     | Potato___healthy         |
|